In [ ]:
# Q1
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15 # training 반복 횟수
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False) # 결과값을 무작위로 보지 않고 순서대로 보기 위해 shuffle=False

In [ ]:
# Q2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias=True).to(device) # 28 * 28 = 784 이기 때문에 1차원에서는 size of input이 784이 된다
#torch.nn.init.normal_(linear.weight)

In [ ]:
# Q3
# Loss fn - Cross Entropy Loss

criterion = torch.nn.CrossEntropyLoss().to(device)

# optimizer - SGD

optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [ ]:
# Q4
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)
        loss = criterion(outputs, labels)

        # optimizer zero gradient 구현
        optimizer.zero_grad()
        # loss backward 구현
        loss.backward()
        # optimizer step 구현
        optimizer.step()

        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))

In [ ]:
# Q5
# test set을 활용해 모델 테스트
linear.eval()

# 해당 범위에서는 gradient 연산x
with torch.no_grad():
    correct = 0
    total = 0

    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)

        outputs = linear(imgs)

        # max()를 통해 최종 출력이 가장 높은 class 선택
        _, argmax = torch.max(outputs, 1)
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()

    print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))